In [31]:
import spotipy

In [32]:
secrets_file = open("..\spoti_secret.txt","r")

In [33]:
string = secrets_file.read()

In [ ]:
string.split('\n')

In [35]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(' ')[0]]=line.split(' ')[1]

In [ ]:
secrets_dict

#### Authentication with secrets

In [37]:
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['csecret']))

#### Searching songs with 'queries' with sp.search

In [38]:
from random import randint
from time import sleep
import numpy as np

def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3000)/1000)
    return tracks

#first we need to get a list of individual tracks from the playlist
def get_name_artist_from_playlist_item(playlist_item):
    return get_name_artists_from_track(playlist_item['track'])

def list_flatten(input_list):
    return [item for sublist in input_list for item in sublist]

def dict_flatten(data, col_list):
    for column in col_list:
        flattened = pd.DataFrame(dict(data[column])).transpose()
        columns = [str(col) for col in flattened.columns]
        flattened.columns = [column + '_' + colname for colname in columns]
        data = pd.concat([data, flattened], axis=1)
        data = data.drop(column, axis=1)
    return data

def get_name_artists_from_playlist(input_playlist):
    return list_flatten(list(map(get_name_artist_from_playlist_item,input_playlist["items"])))

def get_name_artists_from_playlist2(input_playlist):
    return list_flatten(list(map(get_name_artist_from_playlist_item,input_playlist)))

def get_name_artists_from_track(track):
    return [(track["name"],artist["name"]) for artist in track["artists"]]

def get_name_from_track(track):
    return [(track["name"]) for artist in track["artists"]]

def get_artists_from_track(track):
    return [(artist["name"]) for artist in track["artists"]]


In [39]:
# all_tracks = get_playlist_tracks("5S8SJdl1BDc0ugpkEvFsIL")
# len(all_tracks)

In [40]:
# all_tracks[0]['track']['album']

In [41]:
lista = get_playlist_tracks("5S8SJdl1BDc0ugpkEvFsIL")
len(lista)

10000

In [42]:
# lista

In [43]:
import pandas as pd
df = pd.DataFrame(get_name_artists_from_playlist2(lista), columns = ['name','artist'])
df

,name,artist
0,...Ready For It?,Taylor Swift
1,Life Changes,Thomas Rhett
2,24K Magic,Bruno Mars
3,Galway Girl,Ed Sheeran
4,Photograph,Ed Sheeran
...,...,...
12173,Fool's Overture,Supertramp
12174,Heart of the Sunrise - 2003 Remaster,Yes
12175,Venus And Mars / Rock Show / Jet - Live / Rema...,Paul McCartney
12176,Venus And Mars / Rock Show / Jet - Live / Rema...,Wings


#### Borramos las canciones repetidas

In [44]:
df.drop_duplicates(subset = ['name'], keep='first', inplace = True)
# df.drop_duplicates(subset=['brand', 'style'], keep='last')
# df.reset_index(drop=True, inplace=True)
df

,name,artist
0,...Ready For It?,Taylor Swift
1,Life Changes,Thomas Rhett
2,24K Magic,Bruno Mars
3,Galway Girl,Ed Sheeran
4,Photograph,Ed Sheeran
...,...,...
12171,"Remember the Future, Pt. 1",Nektar
12172,Funeral For A Friend / Love Lies Bleeding,Elton John
12173,Fool's Overture,Supertramp
12174,Heart of the Sunrise - 2003 Remaster,Yes


#### Reseteamos los índices

In [45]:
df.reset_index(drop=True, inplace=True)
df

,name,artist
0,...Ready For It?,Taylor Swift
1,Life Changes,Thomas Rhett
2,24K Magic,Bruno Mars
3,Galway Girl,Ed Sheeran
4,Photograph,Ed Sheeran
...,...,...
8846,"Remember the Future, Pt. 1",Nektar
8847,Funeral For A Friend / Love Lies Bleeding,Elton John
8848,Fool's Overture,Supertramp
8849,Heart of the Sunrise - 2003 Remaster,Yes


In [46]:
len(df)

8851

#### Audio_features de Dark Souls 3

In [47]:
# sp.audio_features(lista[0]['track']['uri'])[0]

#### Pasos para crear el dataframe
+ Crear lista de diccionarios
+ Añadirlos al dataframe de artistas
+ Usar flatten para que desenrede los diccionarios

In [48]:
audio_features = []
for i in range(len(df)):
    audio_features.append(sp.audio_features(lista[i]['track']['uri'])[0])

Expected id of type track but found type In+the+Shadow+of+the+Valley spotify:local:::In+the+Shadow+of+the+Valley:187
Expected id of type track but found type April+Sweatpants spotify:local:::April+Sweatpants:166
Expected id of type track but found type Gee+%28Korean+Ver.+MP3+only%29 spotify:local:SNSD::Gee+%28Korean+Ver.+MP3+only%29:205
Expected id of type track but found type Hero spotify:local:::Hero:194
Expected id of type track but found type Beneath+the+Mask+%5BWith+Lyrics%5D+-+Persona+5 spotify:local:::Beneath+the+Mask+%5BWith+Lyrics%5D+-+Persona+5:285
Expected id of type track but found type Last+Surprise+%5BWith+Lyrics%5D+-+Persona+5 spotify:local:::Last+Surprise+%5BWith+Lyrics%5D+-+Persona+5:236
Expected id of type track but found type Vampire+Weekend++Ottoman spotify:local:::Vampire+Weekend++Ottoman:244
Expected id of type track but found type Vampire+Weekend+-+Ottoman spotify:local:::Vampire+Weekend+-+Ottoman:234
Expected id of type track but found type Father+Christmas spot

In [49]:
audio_features = pd.DataFrame({"audio_features":audio_features})
audio_features

,audio_features
0,"{'danceability': 0.615, 'energy': 0.779, 'key'..."
1,"{'danceability': 0.687, 'energy': 0.845, 'key'..."
2,"{'danceability': 0.818, 'energy': 0.803, 'key'..."
3,"{'danceability': 0.624, 'energy': 0.876, 'key'..."
4,"{'danceability': 0.614, 'energy': 0.379, 'key'..."
...,...
8846,"{'danceability': 0.577, 'energy': 0.605, 'key'..."
8847,"{'danceability': 0.498, 'energy': 0.0602, 'key..."
8848,"{'danceability': 0.551, 'energy': 0.324, 'key'..."
8849,"{'danceability': 0.785, 'energy': 0.338, 'key'..."


You have to reset the index to create an actual column with the actual data

In [50]:
audio_features.reset_index(inplace=True)

In [51]:
audio_features

,index,audio_features
0,0,"{'danceability': 0.615, 'energy': 0.779, 'key'..."
1,1,"{'danceability': 0.687, 'energy': 0.845, 'key'..."
2,2,"{'danceability': 0.818, 'energy': 0.803, 'key'..."
3,3,"{'danceability': 0.624, 'energy': 0.876, 'key'..."
4,4,"{'danceability': 0.614, 'energy': 0.379, 'key'..."
...,...,...
8846,8846,"{'danceability': 0.577, 'energy': 0.605, 'key'..."
8847,8847,"{'danceability': 0.498, 'energy': 0.0602, 'key..."
8848,8848,"{'danceability': 0.551, 'energy': 0.324, 'key'..."
8849,8849,"{'danceability': 0.785, 'energy': 0.338, 'key'..."


In [52]:
audio_features = dict_flatten(audio_features, ["audio_features"])
audio_features

,index,audio_features_danceability,audio_features_energy,audio_features_key,audio_features_loudness,audio_features_mode,audio_features_speechiness,audio_features_acousticness,audio_features_instrumentalness,audio_features_liveness,audio_features_valence,audio_features_tempo,audio_features_type,audio_features_id,audio_features_uri,audio_features_track_href,audio_features_analysis_url,audio_features_duration_ms,audio_features_time_signature
0,0,0.615,0.779,2,-6.454,1,0.135,0.0665,0,0.155,0.453,160.0,audio_features,7zgqtptZvhf8GEmdsM2vp2,spotify:track:7zgqtptZvhf8GEmdsM2vp2,https://api.spotify.com/v1/tracks/7zgqtptZvhf8...,https://api.spotify.com/v1/audio-analysis/7zgq...,208198,4
1,1,0.687,0.845,7,-4.37,1,0.0576,0.1,0,0.0452,0.809,87.972,audio_features,4Vxu50qVrQcycjRyJQaZLC,spotify:track:4Vxu50qVrQcycjRyJQaZLC,https://api.spotify.com/v1/tracks/4Vxu50qVrQcy...,https://api.spotify.com/v1/audio-analysis/4Vxu...,190227,4
2,2,0.818,0.803,1,-4.282,1,0.0797,0.034,0,0.153,0.632,106.97,audio_features,6b8Be6ljOzmkOmFslEb23P,spotify:track:6b8Be6ljOzmkOmFslEb23P,https://api.spotify.com/v1/tracks/6b8Be6ljOzmk...,https://api.spotify.com/v1/audio-analysis/6b8B...,225983,4
3,3,0.624,0.876,9,-3.374,1,0.1,0.0735,0,0.327,0.781,99.943,audio_features,0afhq8XCExXpqazXczTSve,spotify:track:0afhq8XCExXpqazXczTSve,https://api.spotify.com/v1/tracks/0afhq8XCExXp...,https://api.spotify.com/v1/audio-analysis/0afh...,170827,4
4,4,0.614,0.379,4,-10.48,1,0.0476,0.607,0.000464,0.0986,0.201,107.989,audio_features,1HNkqx9Ahdgi1Ixy2xkKkL,spotify:track:1HNkqx9Ahdgi1Ixy2xkKkL,https://api.spotify.com/v1/tracks/1HNkqx9Ahdgi...,https://api.spotify.com/v1/audio-analysis/1HNk...,258987,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8846,8846,0.577,0.605,2,-7.422,1,0.0346,0.439,0,0.14,0.37,114.935,audio_features,6SEBRJ6qYR3mnKJivWH1mW,spotify:track:6SEBRJ6qYR3mnKJivWH1mW,https://api.spotify.com/v1/tracks/6SEBRJ6qYR3m...,https://api.spotify.com/v1/audio-analysis/6SEB...,225493,4
8847,8847,0.498,0.0602,3,-20.814,1,0.0682,0.776,0,0.144,0.286,79.382,audio_features,60llRR1nZqjqGeUmfq700K,spotify:track:60llRR1nZqjqGeUmfq700K,https://api.spotify.com/v1/tracks/60llRR1nZqjq...,https://api.spotify.com/v1/audio-analysis/60ll...,208907,4
8848,8848,0.551,0.324,0,-10.962,1,0.0314,0.896,0.0873,0.277,0.231,90.957,audio_features,3JsA2sWDNR9oQogGAzqqtH,spotify:track:3JsA2sWDNR9oQogGAzqqtH,https://api.spotify.com/v1/tracks/3JsA2sWDNR9o...,https://api.spotify.com/v1/audio-analysis/3JsA...,217053,4
8849,8849,0.785,0.338,10,-12.102,0,0.0736,0.421,0.000104,0.331,0.707,83.426,audio_features,0L1lHepiSq6BSDKdO52s6k,spotify:track:0L1lHepiSq6BSDKdO52s6k,https://api.spotify.com/v1/tracks/0L1lHepiSq6B...,https://api.spotify.com/v1/audio-analysis/0L1l...,214133,4


In [53]:
final_df = pd.concat([df, audio_features],axis=1)
final_df

,name,artist,index,audio_features_danceability,audio_features_energy,audio_features_key,audio_features_loudness,audio_features_mode,audio_features_speechiness,audio_features_acousticness,...,audio_features_liveness,audio_features_valence,audio_features_tempo,audio_features_type,audio_features_id,audio_features_uri,audio_features_track_href,audio_features_analysis_url,audio_features_duration_ms,audio_features_time_signature
0,...Ready For It?,Taylor Swift,0,0.615,0.779,2,-6.454,1,0.135,0.0665,...,0.155,0.453,160.0,audio_features,7zgqtptZvhf8GEmdsM2vp2,spotify:track:7zgqtptZvhf8GEmdsM2vp2,https://api.spotify.com/v1/tracks/7zgqtptZvhf8...,https://api.spotify.com/v1/audio-analysis/7zgq...,208198,4
1,Life Changes,Thomas Rhett,1,0.687,0.845,7,-4.37,1,0.0576,0.1,...,0.0452,0.809,87.972,audio_features,4Vxu50qVrQcycjRyJQaZLC,spotify:track:4Vxu50qVrQcycjRyJQaZLC,https://api.spotify.com/v1/tracks/4Vxu50qVrQcy...,https://api.spotify.com/v1/audio-analysis/4Vxu...,190227,4
2,24K Magic,Bruno Mars,2,0.818,0.803,1,-4.282,1,0.0797,0.034,...,0.153,0.632,106.97,audio_features,6b8Be6ljOzmkOmFslEb23P,spotify:track:6b8Be6ljOzmkOmFslEb23P,https://api.spotify.com/v1/tracks/6b8Be6ljOzmk...,https://api.spotify.com/v1/audio-analysis/6b8B...,225983,4
3,Galway Girl,Ed Sheeran,3,0.624,0.876,9,-3.374,1,0.1,0.0735,...,0.327,0.781,99.943,audio_features,0afhq8XCExXpqazXczTSve,spotify:track:0afhq8XCExXpqazXczTSve,https://api.spotify.com/v1/tracks/0afhq8XCExXp...,https://api.spotify.com/v1/audio-analysis/0afh...,170827,4
4,Photograph,Ed Sheeran,4,0.614,0.379,4,-10.48,1,0.0476,0.607,...,0.0986,0.201,107.989,audio_features,1HNkqx9Ahdgi1Ixy2xkKkL,spotify:track:1HNkqx9Ahdgi1Ixy2xkKkL,https://api.spotify.com/v1/tracks/1HNkqx9Ahdgi...,https://api.spotify.com/v1/audio-analysis/1HNk...,258987,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8846,"Remember the Future, Pt. 1",Nektar,8846,0.577,0.605,2,-7.422,1,0.0346,0.439,...,0.14,0.37,114.935,audio_features,6SEBRJ6qYR3mnKJivWH1mW,spotify:track:6SEBRJ6qYR3mnKJivWH1mW,https://api.spotify.com/v1/tracks/6SEBRJ6qYR3m...,https://api.spotify.com/v1/audio-analysis/6SEB...,225493,4
8847,Funeral For A Friend / Love Lies Bleeding,Elton John,8847,0.498,0.0602,3,-20.814,1,0.0682,0.776,...,0.144,0.286,79.382,audio_features,60llRR1nZqjqGeUmfq700K,spotify:track:60llRR1nZqjqGeUmfq700K,https://api.spotify.com/v1/tracks/60llRR1nZqjq...,https://api.spotify.com/v1/audio-analysis/60ll...,208907,4
8848,Fool's Overture,Supertramp,8848,0.551,0.324,0,-10.962,1,0.0314,0.896,...,0.277,0.231,90.957,audio_features,3JsA2sWDNR9oQogGAzqqtH,spotify:track:3JsA2sWDNR9oQogGAzqqtH,https://api.spotify.com/v1/tracks/3JsA2sWDNR9o...,https://api.spotify.com/v1/audio-analysis/3JsA...,217053,4
8849,Heart of the Sunrise - 2003 Remaster,Yes,8849,0.785,0.338,10,-12.102,0,0.0736,0.421,...,0.331,0.707,83.426,audio_features,0L1lHepiSq6BSDKdO52s6k,spotify:track:0L1lHepiSq6BSDKdO52s6k,https://api.spotify.com/v1/tracks/0L1lHepiSq6B...,https://api.spotify.com/v1/audio-analysis/0L1l...,214133,4


In [55]:
# final_df.to_csv("VideojuegosBienChingones.csv")
final_df.to_csv("ten_thousand_songs.csv")
# audio_features.to_csv("audio_features.csv", index=False)